In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
import numpy as np
import tensorflow as tf

In [ ]:
# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Convert the images to RGB (3 channels)
x_train = np.stack((x_train,) * 3, axis=-1)
x_test = np.stack((x_test,) * 3, axis=-1)

# Resize the images to 48x48 pixels (the input size of the VGG16 model)
x_train = tf.image.resize(x_train, (48, 48))
x_test = tf.image.resize(x_test, (48, 48))

# Convert the labels to one-hot encoding
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

x_train.shape

In [ ]:
model = tf.keras.models.Sequential(
    [
        VGG16(weights='imagenet', include_top=False, pooling='avg'),
        Dense(512, activation='relu'),
        Dense(10, activation='softmax')
    ]
)

model.layers[0].trainable = False

In [ ]:
model.summary()

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
y_train.shape

In [ ]:
# Train the model
model.fit(x_train, y_train, epochs=5)